In [5]:
import pandas as pd
data = pd.read_csv('train.csv')
data.head()
# do it on token level
# finer search

,id,raw_address,POI/street
0,0,jl kapuk timur delta sili iii lippo cika 11 a ...,/jl kapuk timur delta sili iii lippo cika
1,1,"aye, jati sampurna",/
2,2,setu siung 119 rt 5 1 13880 cipayung,/siung
3,3,"toko dita, kertosono",toko dita/
4,4,jl. orde baru,/jl. orde baru


In [6]:
raw = data['raw_address'].to_list()
poi_st = data['POI/street'].to_list()
poi = [i.split('/')[0] for i in poi_st]
st = [i.split('/')[1] for i in poi_st]
print(raw[:5], poi[:5], st[:5])

['jl kapuk timur delta sili iii lippo cika 11 a cicau cikarang pusat', 'aye, jati sampurna', 'setu siung 119 rt 5 1 13880 cipayung', 'toko dita, kertosono', 'jl. orde baru'] ['', '', '', 'toko dita', ''] ['jl kapuk timur delta sili iii lippo cika', '', 'siung', '', 'jl. orde baru']


In [7]:
# check if the start of poi/street must be in the raw address
inspect = []
for i, j ,k in zip(raw, poi, st):
    if j != '':
        if j.split()[0] not in i:
            inspect.append((i, j))
    if k != '':
        if k.split()[0] not in i:
            inspect.append((i, k))
print(len(inspect)) # minimize such mismatches

28278


In [8]:
inspect_poi_not_in = []
inspect_poi_in = []
inspect_poi_empty = []
inspect_st_not_in = []
inspect_st_in = []
inspect_st_empty = []

for i, j ,k in zip(raw, poi, st):
    if j == '':
        inspect_poi_empty.append(j)
    elif j not in i:
        inspect_poi_not_in.append(j)
    else:
        inspect_poi_in.append(j)
    if k == '':
        inspect_st_empty.append(k)
    elif k not in i:
        inspect_st_not_in.append(k)
    else:
        inspect_st_in.append(k)
print(len(inspect_poi_empty), len(inspect_poi_not_in), len(inspect_poi_in)) # minimize such mismatches
print(len(inspect_st_empty), len(inspect_st_not_in), len(inspect_st_in)) # minimize such mismatches

178509 46140 75351
70143 17387 212470


In [4]:
inspect = []
for i, j ,k in zip(raw, poi, st):
    if j not in i:
        inspect.append((i, j))
    if k not in i:
        inspect.append((i, k))
print(len(inspect)) # minimize such mismatches

63527


In [5]:
# df_insp = pd.DataFrame({'raw': [i[0] for i in inspect], 'p/s': [i[1] for i in inspect]})
# df_insp.to_csv('inspect.csv', index=False)
inspect[60000]

('smp kris santo elias, mawar,', 'smp kristen santo elias')

In [6]:
lst = []
lst.extend([1]*3)
lst.extend([2]*2)
lst

[1, 1, 1, 2, 2]

In [7]:
spell_dict_right_to_wrong ={}
# construct wrong_right pairs
w_data = []
r_label = []
w_sin = []
for i, j in inspect[:]:
    w_r = []
    i_words = i.split()
    for _j in j.split():
        for _i in i_words:
            if _i in _j and _i != _j and len(_i) > 1: # ignore single char
                w_r = [' '.join([_i if x == _j else x for x in j.split()]),j]
                if w_r[0] and w_r[1]:
                    w_data.append(w_r[0])
                    r_label.append(w_r[1])
                if len(_i) == 1:
                    w_sin.append(w_r)
                if _j in spell_dict_right_to_wrong:
                    if _i in spell_dict_right_to_wrong[_j]:
                        spell_dict_right_to_wrong[_j][_i] +=1
                    else:
                        spell_dict_right_to_wrong[_j][_i] = 1
                else:
                    spell_dict_right_to_wrong[_j] = {_i:1}
                break

In [8]:
w_data[1376] = w_data[1375]
r_label[1376] = r_label[1375]

In [9]:
for i, j ,k in zip(raw, poi, st):
    if j in i:
        if j != '':
            w_data.append(j)
            r_label.append(j)
    if k not in i:
        if k != '':
            w_data.append(k)
            r_label.append(k)

In [10]:
import random
c = list(zip(w_data, r_label))
random.shuffle(c)
w, r = zip(*c)

In [11]:
df_sc = pd.DataFrame({'data': w, 'label': r})
df_sc.to_csv('new_sc.csv', index=True)

In [79]:
# clean spell_dict_right_to_wrong
new_spell_dict_right_to_wrong = {} # sort large to small
for k, v in spell_dict_right_to_wrong.items():
    new_spell_dict_right_to_wrong[k] = {_k: _v for _k, _v in sorted(v.items(), key=lambda item: item[1], reverse=True)} 

In [80]:
d_2 = {}
for k, v in new_spell_dict_right_to_wrong.items():
    d_3 = {}
    for _k, _v in v.items():
        if len(_k) > 1 and _v > 1 and _k != 'nan': # do not map back to single char; do not map back if occurance is too low
            d_3[_k] = _v
    if len(d_3) > 0: 
        d_2[k] = d_3
len(d_2)

4654

In [81]:
d_2['sennaart']

KeyError: 'sennaart'

In [82]:
# d_2 to list
d_4 = {}
for k, v in d_2.items():
    temp_list = []
    for _k, _v in v.items():
        temp_list.extend([_k]*_v)
    d_4[k] = temp_list
len(d_4)

4654

In [83]:
# construct training samples
import random

labels = [i[1] for i in inspect]
data = []
for label in labels:
    new_l = [random.choice(d_4[l]) if l in d_4 else l for l in label.split()]
    new_l_ = ' '.join(new_l)
    data.append(new_l_)

In [84]:
# data[:10] inspect[:10]

In [85]:
# check how is the map back
num_miss = 0
miss = []
for i, j in zip(inspect, data):
    if j not in i[0]:
        num_miss += 1
        miss.append([i[0], i[1], j])
num_miss

30650

In [86]:
miss[:10]

[['yaya atohar,', 'yayasan atohariyah', 'yaya atohariyah'],
 ['mar tabl metro iringmulyo metro timur',
  'markaz tabligh metro',
  'markaz tabligh metro'],
 ['rumah makan pela, raya jomb,', 'rumah makan pelangi', 'rumah mak pela'],
 ['cak 11 nagasari karawang barat', 'cakrad', 'cakrad'],
 ['rnd prin, gang pinak, sukarame', 'rnd printing', 'rnd print'],
 ['pp minhajutt, kh abdul manan, sumberberas muncar',
  'pp minhajutthollab',
  'pp minhajutthollab'],
 ['cau terr, gal,', 'causal terrace', 'causal terrace'],
 ['kedai tenun jep senn, kota bumi, kebon melati',
  'kedai tenun jepara sennaart',
  'kedai tenun jep sennaart'],
 ['cv. hin oto kenc, sido ii, q 29',
  'cv. hingdi oto kencana',
  'cv. hingdi oto kenc'],
 ['caval pizz, dip ukur, lebak gede coblong',
  'cavallino pizzaria',
  'cavallino pizzaria']]

In [87]:
spell_check_dict = {}
for i, j in inspect[:]:
    i_words = i.split()
    for _j in j.split():
        for _i in i_words:
            if _i in _j and _i != _j:
                if _i in spell_check_dict:
                    if _j in spell_check_dict[_i]:
                        spell_check_dict[_i][_j] +=1
                    else:
                        spell_check_dict[_i][_j] = 1
                else:
                    spell_check_dict[_i] = {_j:1}
                break

In [88]:
new_spell_dict = {} # sort large to small
for k, v in spell_check_dict.items():
    new_spell_dict[k] = {_k: _v for _k, _v in sorted(v.items(), key=lambda item: item[1], reverse=True)} 

In [7]:
# finer search by dict entry
train = pd.read_csv('train.csv')
_id = train['id'].to_list()

final_d = {}
for _k,v in new_spell_dict.items():
    first_key = list(v.keys())[0]
    # use this key pair to consturct new raw address
    new_raw = []
    num_change = 0
    for r in raw:
        new_r = [first_key if _r == _k else _r for _r in r.split()]
        _new_r = ' '.join(new_r)
        new_raw.append(_new_r)
    # count how many labels are not in addresses
    inspect = []
    for i, j ,k in zip(new_raw, poi, st):
        if j not in i:
            inspect.append((i, j))
        if k not in i:
            inspect.append((i, k))
    if len(inspect) <= 63527:
        # keep this map
        final_d[_k] = first_key
        print(f'{len(inspect)}, keep {_k}, {first_key}, {str(v[first_key])}')
    else:
        # discard it
        print(f'{len(inspect)}, discard {_k}, {first_key}, {str(v[first_key])}')

63085, keep neg, negeri, 1060
63825, discard boj, bojong, 27
63543, discard yaya, yayasan, 157
63701, discard bang, bangunan, 53
63711, discard mar, market, 20
63590, discard tabl, tabloid, 2
63746, discard cak, cakung, 8
63819, discard bint, bintang, 140
63604, discard kah, kaharu, 9
63591, discard cau, causal, 1
63596, discard jep, jepara, 15
63592, discard hin, hingdi, 1
63770, discard dah, dahlia, 18
63591, discard caval, cavallino, 1
63591, discard mangla, manglayang, 2
63367, keep beng, bengkel, 425
63818, discard cipi, cipinang, 20
63556, discard ber, berkah, 114
63589, discard bust, bustanul, 14
63549, discard mand, mandiri, 267
63589, discard prim, primary, 3
63584, discard kuli, kuliner, 17
63602, discard tapa, tapanuli, 4
64502, discard sela, selatan, 117
63582, discard electr, electronic, 14
63446, keep laun, laundry, 216
63592, discard pasi, pasifik, 5
63626, discard kerto, kertosono, 2
63591, discard stati, statistik, 4
63388, keep kan, kantor, 642
64591, discard pang, pa

In [11]:
train = pd.read_csv('train.csv')
_id = train['id'].to_list()

# # search the threshold where mismatches are mininized
# for _i in range(1031,1032, 11):
#     # filter the dict
#     new_d = {}
#     for k,v in new_spell_dict.items():
#         first_key = list(v.keys())[0]
#         if v[first_key] >= _i:
#             new_d[k] = first_key
    
#     # use this new d to consturct new raw address
#     new_raw = []
#     num_change = 0
#     for r in raw:
#         new_r = [new_d[_r] if _r in new_d.keys() else _r for _r in r.split()]
#         _new_r = ' '.join(new_r)
#         new_raw.append(_new_r)
#     # count how many labels are not in addresses
#     inspect = []
#     for i, j ,k in zip(new_raw, poi, st):
#         if j not in i:
#             inspect.append((i, j))
#         if k not in i:
#             inspect.append((i, k))
#     print(_i, len(inspect)) # minimize such mismatches

In [12]:
final_d

{'neg': 'negeri',
 'beng': 'bengkel',
 'laun': 'laundry',
 'kan': 'kantor',
 'war': 'warung',
 'mush': 'musholla',
 'collec': 'collection',
 'indon': 'indonesia',
 'cah': 'cahaya',
 'kli': 'klinik',
 'apo': 'apotek',
 'fash': 'fashion'}

In [13]:
# construct new train and test
train = pd.read_csv('train.csv')
_id = train['id'].to_list()
raw = train['raw_address'].to_list()
poi_st = train['POI/street'].to_list()
new_raw = []
num_change = 0
for r in raw:
    new_r = [final_d[_r] if _r in final_d.keys() else _r for _r in r.split()]
    _new_r = ' '.join(new_r)
    if _new_r != r:
        num_change += 1
    new_raw.append(_new_r)
print(num_change)
df_train = pd.DataFrame({'id':_id, 'raw_address': new_raw, 'POI/street': poi_st})
df_train.to_csv('new_train_2.csv', index=False)
# construct new train and test
test = pd.read_csv('test.csv')
_id = test['id'].to_list()
raw = test['raw_address'].to_list()
new_raw = []
num_change = 0
for r in raw:
    new_r = [final_d[_r] if _r in final_d.keys() else _r for _r in r.split()]
    _new_r = ' '.join(new_r)
    if _new_r != r:
        num_change += 1
    new_raw.append(_new_r)
print(num_change)
df_test = pd.DataFrame({'id':_id, 'raw_address': new_raw})
df_test.to_csv('new_test_2.csv', index=False)

8449
1394


In [10]:
# 63527 to 60846

new_raw = []
num_change = 0
for r in raw:
    new_r = [final_d[_r] if _r in final_d else _r for _r in r.split()]
    _new_r = ' '.join(new_r)
    new_raw.append(_new_r)
# count how many labels are not in addresses
inspect = []
for i, j ,k in zip(new_raw, poi, st):
    if j not in i:
        inspect.append((i, j))
    if k not in i:
        inspect.append((i, k))
print(len(inspect))

60846


In [ ]:
# remove all the occrance <= 5 dict entries
# also remove if the only difference is adding a dot
def is_sig(k1, k2):
    if len(k2) == len(k1) + 1 and '.' in k2:
        return False
    else:
        return True
spell_dict_trimed = {}
for k, v in new_spell_dict.items():
    new_v = {}
    for _k, _v in v.items():
        if _v > 5:
#         if _v > 1 or len(v) == 1:
            # check if the difference is significant
            if is_sig(k, _k):
                new_v[_k] = _v
    if len(new_v) > 0:
        spell_dict_trimed[k] = new_v

In [ ]:
for k in list(spell_dict_trimed.keys())[:5]:
    print(k, spell_dict_trimed[k])

In [ ]:
spell_dict_trimed['jl'] # should not be there

In [ ]:
# sum the first match and all
num_first = 0
num_all = 0
for k, v in spell_dict_trimed.items():
    num_first += v[list(v.keys())[0]]
    for _k, _v in v.items():
        num_all += _v
# 're mar' -> 're martad'
print(num_first/num_all, len(spell_dict_trimed))

In [ ]:
# keep only the first for processing

# do not update if the only difference is a punctualation

def hasNumbers(inputString):
    return any(char.isdigit() for char in inputString)
spell_dict_trimed_3 = {}
for k, v in spell_dict_trimed.items():
    if hasNumbers(k):
        continue
    for _k, _v in v.items():
        spell_dict_trimed_3[k] = _k
        break

In [ ]:
df_raw_new = pd.DataFrame({'raw_word': list(spell_dict_trimed_3.keys()), 'new_word': list(spell_dict_trimed_3.values())})
df_raw_new.to_csv('raw_word_to_new_word.csv', index=False)

In [ ]:
# construct new train and test
train = pd.read_csv('train.csv')
_id = train['id'].to_list()
raw = train['raw_address'].to_list()
poi_st = train['POI/street'].to_list()
new_raw = []
num_change = 0
for r in raw:
    new_r = [spell_dict_trimed_3[_r] if _r in spell_dict_trimed_3.keys() else _r for _r in r.split()]
    _new_r = ' '.join(new_r)
    if _new_r != r:
        num_change += 1
    new_raw.append(_new_r)
print(num_change)
df_train = pd.DataFrame({'id':_id, 'raw_address': new_raw, 'POI/street': poi_st})
df_train.to_csv('new_train.csv', index=False)

In [ ]:
# construct new train and test
test = pd.read_csv('test.csv')
_id = test['id'].to_list()
raw = test['raw_address'].to_list()
new_raw = []
num_change = 0
for r in raw:
    new_r = [spell_dict_trimed_3[_r] if _r in spell_dict_trimed_3.keys() else _r for _r in r.split()]
    _new_r = ' '.join(new_r)
    if _new_r != r:
        num_change += 1
    new_raw.append(_new_r)
print(num_change)
df_test = pd.DataFrame({'id':_id, 'raw_address': new_raw})
df_test.to_csv('new_test.csv', index=False)

In [ ]:
# num_first_2 = 0
# num_all = 0
# for k, v in spell_dict_trimed.items():
#     keys = list(v.keys())
#     num_first_2 += v[keys[0]]
#     if len(keys) > 1:
#         num_first_2 += v[keys[1]]
#     for _k, _v in v.items():
#         num_all += _v

In [ ]:
# print(num_first_2, num_all)

In [ ]:
# # keep the first 2 only
# # remove numbers 
# spell_dict_trimed_2 = {}
# for k, v in spell_dict_trimed.items():
#     try:
#         int(k)
#     except ValueError:
#         new_v = {}
#         keys = list(v.keys())
#         new_v[keys[0]] = v[keys[0]]
#         if len(keys) > 1:
#             new_v[keys[1]] = v[keys[1]]
#         spell_dict_trimed_2[k] = new_v

In [ ]:
# d_2 = {}
# for k ,v in spell_dict_trimed_2.items():
#     if len(v) == 2:
#         d_2[k] = v

In [ ]:
# list(spell_dict_trimed_3.values())